In [1]:
!pip install datasets evaluate transformers --force-reinstall transformers==4.20.1

  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached transformers-4.29.0-py3-none-any.whl (7.1 MB)
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached requests-2.30.0-py3-none-any.whl (62 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached numpy-1.24.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached tokenizers-0.12.1-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached regex-2023.5.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using ca

In [2]:
import os

# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate

2023-05-10 21:18:48.891944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 21:18:49.579340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv("data/amazon.csv")

for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_csv(os.path.join(root, filename))
        df = pd.concat([df, temp_df], axis=0, sort=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18447 entries, 0 to 668
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          18447 non-null  int64  
 1   Дата                10873 non-null  object 
 2   Время               10873 non-null  object 
 3   Цена до             10873 non-null  float64
 4   Цена после          10873 non-null  float64
 5   Разница в долларах  10873 non-null  float64
 6   Дельта в процентах  10873 non-null  float64
 7   Текст новости       10873 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.3+ MB


In [4]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.drop(columns=[df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"}, inplace=True)

df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["percentage_price_difference"] = df["absolute_price_difference"] / df["price_before"] * 100
df["price_change_direction"] = np.where(df["absolute_price_difference"] > 0, 1, 0)
df = df[["news_text", "price_change_direction"]].copy()
df.head()

,news_text,price_change_direction
0,Check Out What Whales Are Doing With NOC\n,0
1,What 7 Analyst Ratings Have To Say About North...,0
2,7 Analysts Have This to Say About Northrop Gru...,0
3,"Benzinga's Top Ratings Upgrades, Downgrades Fo...",0
5,Looking Into Northrop Grumman's Recent Short I...,1


In [5]:
train_data = df.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = df.drop(train_data.index)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 2874 records.
The testing dataset has 54 records.


In [6]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

In [7]:
print(f'The length of hg_train_data is {len(hg_train_data)}.\n')

# Check one review
hg_train_data[0]

The length of hg_train_data is 2874.



{'news_text': "Will Chart Analysts Notice Bad Omen on Netflix's Chart\n",
 'price_change_direction': 0,
 '__index_level_0__': 426}

In [8]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["news_text"], 
                     max_length=32,
                     truncation=True, 
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/2874 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [10]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['news_text', 'price_change_direction', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2874
})
Dataset({
    features: ['news_text', 'price_change_direction', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 54
})


In [11]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [13]:
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 157 evaluation models in Hugging Face.



['lvwerra/test',
 'precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'jordyvl/ece',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'lvwerra/bary_score',
 'kaggle/amex',
 'kaggle/ai4code',
 'hack/test_metric',
 'yzha/ctc_eval',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'daiyizheng/valid',
 'poseval',
 'erntkn/dice_coefficient',
 'mgfrantz/roc_auc_macro',
 'Vlasta/pr_auc',
 'gorkaartola/metric_for_tp_fp_samples',
 'idsedykh/metric',
 'idsedykh/codebleu2',
 'idsed

In [14]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, price_change_direction, news_text. If __index_level_0__, price_change_direction, news_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/alexa/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2874
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 180


ValueError: Target size (torch.Size([32, 32])) must be the same as input size (torch.Size([32, 2]))